# Data Preparation
This notebook contains all data preparation and preprocessing steps for the yeast ML project.

## 1. Import Libraries

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer

## 2. Load Dataset

In [3]:
# Load the raw dataset
data = pd.read_csv('yeast.csv')



## 3. Data Cleaning

### 3.1 Handle Missing Values

In [8]:
# Check for missing values
missing_values = data.isnull().sum()
print("Missing values in each column:\n", missing_values)

Missing values in each column:
 SeqName        0
mcg            0
gvh            0
alm            0
mit            0
erl            0
pox            0
vac            0
nuc            0
ProteinSite    0
dtype: int64


In [9]:
# Handle missing values (imputation or removal)


### 3.2 Handle Duplicates

In [10]:
# Remove duplicate rows
data = data.drop_duplicates()

### 3.3 Handle Outliers

In [11]:
# Handle outliers (removal or transformation)


## 4. Feature Engineering

### 4.1 Create New Features

In [ ]:
# Create new features if needed


### 4.2 Feature Transformation

In [ ]:
# Apply transformations (log, sqrt, etc.)


### 4.3 Feature Selection

In [ ]:
# Select relevant features

## 5. Encoding Categorical Variables

### 5.1 Label Encoding

In [ ]:
# Apply label encoding for ordinal variables


### 5.2 One-Hot Encoding

In [ ]:
# Apply one-hot encoding for nominal variables

## 6. Feature Scaling

### 6.1 Standardization

In [ ]:
# Apply standardization (StandardScaler)


### 6.2 Normalization

In [ ]:
# Apply normalization (MinMaxScaler)


## 7. Train-Test Split

In [ ]:
# Split data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

## 8. Handle Class Imbalance (if applicable)

In [ ]:
# Apply techniques like SMOTE, undersampling, or oversampling


## 9. Save Processed Data

In [ ]:
# Save the cleaned and preprocessed data
train_data.to_csv('preprocessed_train.csv', index=False)
test_data.to_csv('preprocessed_test.csv', index=False)

## 10. Summary

### Data preparation summary:
-
-
-